In [1]:
import os,json
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten, Input, Conv1D,MaxPooling1D,Embedding
from keras.models import load_model, Model
from keras.utils import to_categorical
from sklearn.metrics import classification_report
import pandas as pd
import sqlalchemy as sa
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import gensim
from nltk.corpus import stopwords
import collections
from collections import defaultdict
import nltk
user = 'dev_ds'
pwd = 'akBefvuIj4'
from base64 import b64decode
conn_str = 'mssql+pyodbc://%s:%s@repdb1irv.iherb.net/iHerb_Shop?driver=ODBC+Driver+13+for+SQL+Server'
engine = sa.create_engine(conn_str % ('dev_ds', b64decode('YWtCZWZ2dUlqNA==').decode()))

import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

C:\Users\fjavadi\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\fjavadi\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [198]:
#model=gensim.models.word2vec(x,size=100)

In [199]:
#Removing the main first level categories
#ParentCategoryID=8120 : Brands, HealthTopics=
p='inventoryvelocity'
query3='''
SELECT A.ProductID as ProductID, A.ParentCategoryID as CategoryID, B.level as level, B.d as d, B.Name as Name 
FROM `inventoryvelocity.input.product_parent_categories_relation` AS A
Inner join `rec-serv.AutoSuggestion.category_data` AS B on A.ParentCategoryID=B.CategoryID 
Where B.VisibleToCustomer=True and B.CategoryID not in (1475,1855,2089,2203,2236,2320,2992,3213,100477,100483,2430,2493,2465,8120)
and B.ParentCategoryID not in (8120,2430)
'''
products_OldCategories=pd.read_gbq(query3,p,dialect='standard')

product_attr = pd.read_sql(''' 
SELECT [ProductID]
      ,[DisplayName]
      ,[Ingredients]
      ,[Description]
      ,[SupplementFacts]
      ,[ServingSize]
      ,[PackageQuantity]
      ,[SuggestedUse]
      ,[Keywords]
  FROM [iHerb_Shop].[shopservice].[TBL_Products]
  where Status in (0,3,4,6)
''', engine)
product_attr.set_index('ProductID' , inplace=True)

product_attr =product_attr[~product_attr.isnull()]
product_attr =product_attr[~product_attr['Description'].isnull()]

product_attr =product_attr[~product_attr.isnull()]
product_attr =product_attr[~product_attr['DisplayName'].isnull()]

print(len(product_attr))
products_OldCategories=products_OldCategories[products_OldCategories['ProductID'].isin(product_attr.index.values)]
print(products_OldCategories['ProductID'].nunique())

30118
29573


In [200]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', ' ', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isnumeric() or isinstance(word, float):
            new_word=' '
            #new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
           # word1 = re.sub(r'[0-9]+', '', word)
            word1=word
            new_words.append(word1)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        #lemma = lemmatizer.lemmatize(word, pos='v')
        lemma = lemmatizer.lemmatize(word)
        if(lemma!=" "):
            lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
   # words = remove_stopwords(words)
    #words=stem_words(words)
    words= lemmatize_verbs(words)
    return words



In [201]:
remove_punctuation("caffeine-free")

['c', 'a', 'f', 'f', 'e', 'i', 'n', 'e', ' ', 'f', 'r', 'e', 'e']

In [202]:
stops = set(stopwords.words("english"))
def preprocess(text):
    words = [w for w in text.lower().split() if not w in stops]
    final_text = " ".join(words)
    final_text = re.sub(r"[^A-Za-z0-9(),!.?\'`]", " ", final_text )
    final_text = re.sub(r"\'s", " 's ", final_text )
    final_text = re.sub(r"\'ve", " 've ", final_text )
    final_text = re.sub(r"n\'t", " 't ", final_text )
    final_text = re.sub(r"\'re", " 're ", final_text )
    final_text = re.sub(r"\'d", " 'd ", final_text )
    final_text = re.sub(r"\'ll", " 'll ", final_text )
    final_text = re.sub(r",", " ", final_text )
    final_text = re.sub(r"\.", " ", final_text )
    final_text = re.sub(r"!", " ", final_text )
    final_text = re.sub(r"\(", " ( ", final_text )
    final_text = re.sub(r"\)", " ) ", final_text )
    final_text = re.sub(r"\?", " ", final_text )
    final_text = re.sub(r"\s{2,}", " ", final_text )
    return final_text

In [203]:
full_text_tokenized=[]
for index,row in product_attr.iterrows():
    s=product_attr.loc[index]['DisplayName']
    s = re.sub(r'[^\w\s]',' ',s)
    #s = s.translate(None, string.punctuation)
    words = nltk.word_tokenize(s)
   # words = nltk.word_tokenize(product_attr.loc[index]['DisplayName'])
   # words=product_attr.loc[index]['DisplayName'].split()
    words = normalize(words)
   # words = [x for x in words if x != ' ']
    full_text_tokenized.append(words)
   # words1 = nltk.word_tokenize(product_attr.loc[index]['Description'])
   # words1 = normalize(words)
   # words = words + words1
   # full_text_tokenized.append(words)
    
    #full_text_tokenized.append (gensim.utils.simple_preprocess (product_attr.loc[index]['DisplayName']))
    #full_text_tokenized.append (gensim.utils.simple_preprocess (product_attr.loc[index]['Description']))
    
    #full_text_tokenized.append(product_attr.loc[index]['DisplayName'].split(" "))
model = gensim.models.Word2Vec(full_text_tokenized, size=50,sg=1,window=2,min_count=1)
model.train(full_text_tokenized, total_examples=len(full_text_tokenized), epochs=10)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
print(w2v)

KeyboardInterrupt: 

In [150]:
model.wv.most_similar("vitamin")

[('vitashine', 0.8049868941307068),
 ('vit', 0.7961345911026001),
 ('gamma', 0.7811069488525391),
 ('ester', 0.7617003917694092),
 ('d2', 0.7550777196884155),
 ('cholecalciferol', 0.7543282508850098),
 ('calcium', 0.7437788248062134),
 ('famil', 0.7422420382499695),
 ('k', 0.7393620014190674),
 ('mune', 0.7389215230941772)]

In [204]:
#Categories:1475
#HealthTopic: 2430
#"Heart, Cardiovascular Health":2493
query1='''
SELECT * FROM [rec-serv:AutoSuggestion.category_data]

'''
category_data=pd.read_gbq(query1,p)
category_data.set_index('CategoryID' , inplace=True)



In [205]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        #self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        #self.dim = len(word2vec.values().next())

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean( np.array([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec]),axis=0)
                for words in X
            ])

In [153]:
#print(full_text_tokenized)
print(w2v['tea'])
print(len(w2v))

[-0.5495944  -0.23383164  1.0819181  -0.14568366  0.26505804 -0.71710503
  0.8807161   0.1832275   0.07973384 -0.1867346   0.19036809 -0.06379269
  0.7285378   0.01743434 -0.50183356  0.20933518 -0.95880884  0.02499827
  1.4527898   0.7873222  -1.0249469   0.01113395  0.2872568  -1.0185341
 -0.6304571   0.9445531  -1.1772006   0.21971816  0.3592905   0.15764579
 -1.1546912   0.26406804 -0.62077206 -0.53981006 -0.57684386 -0.99948245
  0.41636598  0.4413555  -0.01852245 -0.31816018 -0.22889566  0.33548093
 -1.573684    0.5987424  -1.0001405  -0.66046154  1.2885921  -0.8399659
 -0.21379451  0.71518075]
8136


In [154]:
print(full_text_tokenized)

[['htp', 'mg', 'veggie', 'cap'], ['alpha', 'lipoic', 'acid', 'mg', 'veggie', 'capsule'], ['msm', 'with', 'optimsm', 'mg', 'tablet'], ['stabilized', 'r', 'lipoic', 'acid', 'with', 'bioenhanced', 'na', 'rala', 'mg', 'veggie', 'cap'], ['msm', 'powder', 'with', 'optimsm', 'oz', 'g'], ['ultra', 'cordyceps', 'mg', 'veggie', 'cap'], ['quercetin', 'bromelain', 'veggie', 'cap'], ['htp', 'enhanced', 'with', 'vitamin', 'b6', 'c', 'veggie', 'cap'], ['glucosamine', 'sulfate', 'mg', 'capsule'], ['ultra', 'cordyceps', 'plus', 'veggie', 'cap'], ['digestive', 'enzyme', 'veggie', 'cap'], ['curcumin', 'high', 'absorption', 'mg', 'capsule'], ['high', 'absorption', 'magnesium', 'chelated', 'with', 'albion', 'mineral', 'mg', 'tablet'], ['french', 'red', 'wine', 'grape', 'extract', 'mg', 'veggie', 'cap'], ['msm', 'with', 'optimsm', 'mg', 'capsule'], ['high', 'absorption', 'coq10', 'with', 'bioperine', 'mg', 'veggie', 'cap'], ['synergistic', 'glucosamine', 'msm', 'formula', 'with', 'optimsm', 'capsule'], ['gl

In [155]:
sentences=[]
for w in full_text_tokenized:
    sentence=" ".join(w)
    sentences.append(sentence)
#print(sentences)
if(" " in sentences):
    print("1")

In [156]:
#Full_text_list=product_attr['Description'].values.tolist()
#Full_text_list=Full_text_list.append(product_attr['DisplayName'].values.tolist())

model_TfidfEmbeddingVectorizer=TfidfEmbeddingVectorizer(w2v)
#model_TfidfEmbeddingVectorizer.fit(Full_text_list)
model_TfidfEmbeddingVectorizer.fit(product_attr['DisplayName'])
#model_TfidfEmbeddingVectorizer.fit(sentences)
tfidfEmbedding_matrix_allProducts=model_TfidfEmbeddingVectorizer.transform(full_text_tokenized)
print(tfidfEmbedding_matrix_allProducts.shape)
cosine_similarities=np.zeros((tfidfEmbedding_matrix_allProducts.shape[0],tfidfEmbedding_matrix_allProducts.shape[0]))
cosine_similarities=cosine_similarity(tfidfEmbedding_matrix_allProducts,tfidfEmbedding_matrix_allProducts)
#cosine_similarities=np.asarray(cosine_similarity(tfidfEmbedding_matrix_allProducts,tfidfEmbedding_matrix_allProducts))

(30117, 50)


In [157]:
print(cosine_similarities.shape)

(30117, 30117)


In [158]:
product_similarities=pd.DataFrame(cosine_similarities, index=product_attr.index.values.tolist(),columns=product_attr.index.values.tolist())
product_similar_dict = dict()
for product in product_attr.index.tolist():
    product_similar_dict[product] = product_similarities.loc[product].sort_values(ascending=False).index.tolist()[1:11]

In [206]:
print(products_OldCategories[products_OldCategories['level']==1].nunique())
products_level0=products_OldCategories[products_OldCategories['level']==0]
products_level1=products_OldCategories[products_OldCategories['level']==1]
products_level2=products_OldCategories[products_OldCategories['level']==2]
products_level3=products_OldCategories[products_OldCategories['level']==3]
products_level4=products_OldCategories[products_OldCategories['level']==4]
products_level5=products_OldCategories[products_OldCategories['level']==5]


ProductID     5010
CategoryID      38
level            1
d               38
Name            38
dtype: int64


In [207]:
def find_category(p,products_level):
    if p in products_level['ProductID'].values.tolist():
        df=products_level[products_level['ProductID']==p]
        return df['CategoryID'].values.tolist()
    else:
        return [0]
    

In [185]:
extra_level0_products=[i for i in product_attr.index.values if i not in products_level0['ProductID'].values.tolist()]
extra_level1_products=[i for i in product_attr.index.values if i not in products_level1['ProductID'].values.tolist()]
extra_level2_products=[i for i in product_attr.index.values if i not in products_level2['ProductID'].values.tolist()]
extra_level3_products=[i for i in product_attr.index.values if i not in products_level3['ProductID'].values.tolist()]
extra_level4_products=[i for i in product_attr.index.values if i not in products_level4['ProductID'].values.tolist()]
extra_level5_products=[i for i in product_attr.index.values if i not in products_level5['ProductID'].values.tolist()]

In [186]:
for product in extra_level1_products:
    print(product)
    nested_categories= [find_category(x,products_level1) for x in product_similar_dict[product]]
    flat_categories = [item for sublist in nested_categories for item in sublist]
   # print(flat_categories)
    most_commons= Counter(flat_categories).most_common(5)
    print(most_commons)
    

27
[(0, 5), (2217, 3), (2432, 2), (2443, 1), (2237, 1)]
30
[(2489, 6), (2018, 6), (2455, 5), (0, 3), (2480, 2)]
42
[(2217, 9), (2219, 9), (2267, 5), (2415, 5), (2439, 5)]
64
[(0, 8), (2091, 1), (2431, 1), (2452, 1), (2454, 1)]
106
[(0, 6), (2438, 3), (2091, 1), (2431, 1), (2432, 1)]
108
[(0, 7), (2091, 1), (2431, 1), (2452, 1), (2454, 1)]
111
[(0, 7), (2091, 1), (2431, 1), (2452, 1), (2454, 1)]
112
[(0, 7), (2494, 2), (2432, 1), (1935, 1), (2438, 1)]
113
[(101046, 10), (3485, 2), (7295, 2)]
114
[(4224, 8), (2438, 8), (0, 2), (7517, 1), (2432, 1)]
116
[(0, 7), (2456, 3), (101046, 3), (2495, 3), (7295, 1)]
118
[(101046, 3), (2455, 3), (2452, 2), (2489, 2), (2432, 1)]
119
[(2438, 4), (0, 3), (1618, 2), (2472, 2), (2091, 2)]
120
[(0, 5), (2494, 2), (2246, 2), (2438, 1), (2455, 1)]
124
[(0, 8), (2443, 2), (2415, 1), (2439, 1), (2455, 1)]
126
[(0, 6), (101046, 2), (2237, 1), (2464, 1), (2267, 1)]
128
[(0, 5), (2439, 3), (2443, 2), (2237, 2), (2436, 1)]
130
[(2455, 5), (2489, 5), (2018, 5), (

[(2432, 3), (2217, 3), (0, 3), (7295, 1), (2415, 1)]
615
[(101046, 6), (0, 3), (2480, 1), (2415, 1), (2439, 1)]
632
[(0, 5), (2246, 3), (2451, 2), (2091, 1), (2431, 1)]
633
[(0, 4), (2217, 3), (2432, 2), (2219, 1), (2464, 1)]
640
[(0, 3), (2246, 2), (2267, 2), (2480, 2), (2217, 2)]
653
[(101046, 5), (0, 4), (2480, 1), (2415, 1), (2439, 1)]
657
[(101046, 5), (0, 4), (1935, 2), (2489, 2), (2481, 1)]
661
[(0, 6), (101046, 3), (2415, 2), (2439, 2), (1935, 2)]
667
[(0, 9), (101046, 1), (2237, 1)]
673
[(0, 6), (101046, 2), (1964, 1), (2007, 1), (2435, 1)]
678
[(0, 6), (101046, 2), (2237, 1), (2464, 1), (2267, 1)]
686
[(0, 5), (2439, 4), (2415, 2), (2489, 1), (2018, 1)]
690
[(0, 4), (2443, 2), (2237, 2), (2455, 2), (2439, 1)]
706
[(4224, 4), (0, 2), (2246, 2), (2494, 2), (2438, 1)]
708
[(0, 8), (2438, 1), (2091, 1), (2431, 1), (2432, 1)]
714
[(0, 8), (1935, 2)]
718
[(0, 6), (2217, 2), (2219, 2), (2188, 1), (1618, 1)]
748
[(0, 5), (2246, 4), (2217, 2), (2219, 2), (2464, 1)]
762
[(0, 4), (4866,

[(2091, 3), (2431, 3), (2452, 3), (2454, 3), (2217, 3)]
1672
[(0, 6), (101046, 2), (2237, 2), (2415, 1), (2439, 1)]
1710
[(2415, 6), (2439, 6), (2217, 5), (2219, 5), (2456, 5)]
1712
[(0, 6), (4866, 1), (2438, 1), (2494, 1), (2451, 1)]
1728
[(0, 8), (7295, 1), (2217, 1), (2219, 1)]
1732
[(0, 8), (7295, 1), (2217, 1), (2219, 1)]
1742
[(2091, 4), (2431, 4), (2452, 3), (2454, 3), (2472, 3)]
1745
[(0, 7), (7517, 2), (2438, 2), (2479, 2), (7295, 1)]
1754
[(2091, 4), (2431, 4), (2452, 4), (2454, 4), (2217, 4)]
1797
[(0, 6), (2246, 4), (2091, 3), (2431, 3), (1935, 3)]
1813
[(2246, 8), (2495, 3), (4224, 2), (2463, 2), (2076, 2)]
1817
[(2219, 5), (2217, 4), (0, 3), (2091, 2), (2431, 2)]
1820
[(0, 5), (2246, 4), (2091, 3), (2431, 3), (2452, 3)]
1821
[(2415, 7), (2439, 7), (2456, 7), (2495, 7), (0, 1)]
1833
[(2091, 5), (2431, 5), (1935, 5), (2452, 5), (2454, 5)]
1837
[(0, 10)]
1839
[(0, 4), (2219, 2), (4224, 2), (2456, 2), (2495, 2)]
1840
[(0, 6), (2246, 4), (2091, 1), (2431, 1), (2452, 1)]
1841
[

[(0, 6), (2436, 1), (2438, 1), (2415, 1), (2439, 1)]
3316
[(2212, 8), (2205, 6), (2091, 5), (2431, 5), (2452, 5)]
3327
[(0, 8), (2476, 2)]
3330
[(0, 10)]
3332
[(0, 9), (2007, 1)]
3337
[(0, 4), (2246, 3), (1964, 2), (2476, 1)]
3344
[(0, 6), (2091, 3), (2431, 3), (2452, 3), (2454, 3)]
3365
[(0, 5), (2246, 2), (2088, 1), (2091, 1), (2431, 1)]
3414
[(0, 8), (2463, 1), (2091, 1), (2431, 1), (2432, 1)]
3453
[(0, 6), (2091, 2), (2217, 2), (2219, 2), (2246, 2)]
3459
[(0, 7), (2213, 1), (2435, 1), (2456, 1), (2210, 1)]
3465
[(0, 10)]
3466
[(0, 10)]
3468
[(0, 10)]
3469
[(0, 10)]
3470
[(0, 10)]
3475
[(0, 10)]
3476
[(0, 10)]
3477
[(0, 6), (7295, 4)]
3479
[(0, 9), (2217, 1)]
3480
[(0, 9), (2217, 1)]
3482
[(0, 9), (2217, 1)]
3484
[(0, 4), (101046, 2), (2415, 2), (2439, 2), (2246, 2)]
3485
[(0, 9), (2217, 1)]
3486
[(0, 9), (2217, 1)]
3488
[(0, 9), (2217, 1)]
3489
[(0, 9), (2217, 1)]
3490
[(0, 9), (2217, 1)]
3497
[(0, 9), (2217, 1)]
3498
[(0, 9), (2217, 1)]
3503
[(0, 9), (2217, 1)]
3509
[(0, 10)]
3510

[(0, 8), (2438, 1), (2478, 1), (2471, 1)]
5040
[(2436, 5), (2438, 5), (2076, 3), (4866, 2), (2246, 2)]
5041
[(0, 10)]
5044
[(0, 9), (2246, 1)]
5056
[(0, 8), (2471, 1), (7295, 1)]
5057
[(0, 5), (2246, 4), (1964, 1)]
5062
[(0, 8), (2091, 2), (2431, 2), (2443, 2), (2452, 2)]
5063
[(0, 8), (2219, 2), (2091, 1), (2431, 1), (2452, 1)]
5068
[(0, 8), (7295, 2), (2246, 1)]
5072
[(0, 9), (2205, 1), (2212, 1)]
5075
[(0, 7), (2471, 1), (1964, 1), (2088, 1), (2091, 1)]
5077
[(0, 9), (7295, 1), (2246, 1)]
5083
[(0, 10)]
5084
[(0, 10)]
5085
[(0, 10)]
5087
[(0, 10)]
5088
[(0, 10)]
5089
[(0, 10)]
5091
[(0, 10)]
5092
[(0, 10)]
5093
[(0, 10)]
5094
[(0, 10)]
5095
[(0, 10)]
5096
[(0, 10)]
5097
[(0, 10)]
5102
[(0, 10)]
5103
[(0, 10)]
5104
[(0, 10)]
5105
[(0, 10)]
5106
[(0, 10)]
5108
[(0, 10)]
5109
[(0, 10)]
5110
[(0, 10)]
5111
[(0, 10)]
5112
[(0, 10)]
5113
[(0, 10)]
5114
[(0, 10)]
5115
[(0, 10)]
5116
[(0, 10)]
5117
[(0, 10)]
5119
[(2415, 7), (2439, 7), (2217, 6), (2219, 6), (2478, 4)]
5130
[(1935, 4), (0, 4

...

[(0, 8), (2175, 1), (101335, 1)]
85926
[(0, 8), (2175, 1), (101335, 1)]
85927
[(0, 8), (2175, 1), (101335, 1)]
85928
[(0, 8), (2175, 1), (101335, 1)]
85932
[(0, 10)]
85933
[(0, 4), (2489, 3), (2448, 1), (2491, 1), (2471, 1)]
85934
[(0, 6), (100529, 2), (2205, 1), (2212, 1), (1935, 1)]
85936
[(0, 3), (2489, 3), (2246, 3), (2448, 1), (2183, 1)]
85937
[(0, 5), (2489, 4), (2478, 3), (2491, 3), (2246, 2)]
85943
[(0, 8), (2471, 1), (1964, 1), (100529, 1)]
85944
[(0, 8), (2471, 1), (1964, 1), (100529, 1)]
85945
[(0, 8), (2471, 1), (1964, 1), (100529, 1)]
85946
[(0, 8), (2471, 1), (1964, 1), (100529, 1)]
85947
[(0, 8), (2476, 1), (100529, 1), (2482, 1)]
85949
[(2480, 9), (2079, 5), (2435, 5), (2456, 5), (2210, 5)]
85950
[(2435, 8), (2456, 8), (2210, 8), (2495, 8), (0, 1)]
85951
[(2246, 3), (7517, 2), (2456, 2), (2495, 2), (2438, 2)]
85952
[(2246, 3), (7517, 2), (2456, 2), (2495, 2), (2438, 2)]
85956
[(2246, 3), (7517, 2), (2456, 2), (2495, 2), (2438, 2)]
85957
[(0, 8), (8410, 1), (2458, 1), (2

[(0, 10)]
86291
[(0, 10)]
86292
[(0, 10)]
86293
[(0, 10)]
86294
[(0, 10)]
86295
[(2439, 8), (2237, 8), (2443, 7), (1935, 6), (2091, 4)]
86296
[(0, 4), (2246, 3), (2217, 3), (2219, 2), (2175, 1)]
86297
[(0, 4), (2246, 3), (2217, 3), (2219, 2), (2175, 1)]
86299
[(0, 3), (2495, 2), (2246, 2), (101046, 2), (4224, 1)]
86300
[(0, 6), (2175, 1), (2489, 1), (6156, 1), (2217, 1)]
86304
[(0, 10)]
86314
[(0, 4), (2246, 3), (2471, 2), (2489, 2), (2088, 1)]
86316
[(2475, 5), (0, 4), (2246, 3), (2188, 2), (1964, 2)]
86317
[(2435, 8), (2456, 7), (2210, 7), (2495, 7), (2079, 3)]
86318
[(2237, 6), (2438, 2), (0, 2)]
86319
[(2205, 6), (2212, 6), (0, 4), (2475, 1), (1964, 1)]
86320
[(0, 8), (2088, 1), (2091, 1), (2431, 1), (3212, 1)]
86321
[(0, 4), (8410, 4), (101231, 2)]
86323
[(0, 10)]
86324
[(0, 10)]
86325
[(0, 8), (2246, 1), (1964, 1)]
86326
[(0, 10)]
86327
[(0, 10)]
86328
[(0, 9), (2471, 1)]
86329
[(0, 10)]
86330
[(0, 9), (2482, 1)]
86331
[(0, 10)]
86332
[(0, 10)]
86333
[(0, 10)]
86334
[(0, 10)]
863

[(0, 7), (2205, 2), (2212, 2), (101046, 1)]
86634
[(101046, 7), (2246, 4), (0, 3), (4574, 1)]
86636
[(6156, 7), (0, 2), (101046, 1)]
86637
[(2217, 5), (2219, 5), (2267, 5), (2091, 4), (2431, 4)]
86639
[(0, 8), (2007, 1), (100529, 1)]
86640
[(2205, 7), (2212, 7), (0, 2), (1964, 2), (2475, 2)]
86641
[(2464, 7), (2246, 7), (2415, 2), (2439, 2), (2456, 2)]
86642
[(0, 8), (8410, 1), (101231, 1)]
86643
[(0, 4), (2415, 1), (2439, 1), (2456, 1), (2495, 1)]
86644
[(0, 10)]
86645
[(2480, 8), (2456, 5), (2495, 5), (2435, 4), (2210, 4)]
86646
[(2456, 4), (2495, 4), (0, 3), (1618, 2), (2438, 2)]
86647
[(2091, 6), (2431, 6), (2452, 6), (2454, 6), (2211, 6)]
86648
[(2415, 6), (2439, 6), (2456, 5), (2495, 5), (2217, 4)]
86649
[(0, 4), (2091, 3), (2246, 2), (2438, 2), (7517, 2)]
86650
[(2246, 3), (7517, 2), (2456, 2), (2495, 2), (2438, 2)]
86651
[(0, 4), (2091, 3), (2431, 3), (2452, 3), (2454, 3)]
86652
[(2091, 4), (2431, 4), (2452, 3), (2454, 3), (2217, 3)]
86653
[(0, 4), (2443, 2), (2237, 2), (2438, 

[(0, 10)]
87087
[(0, 10)]
87088
[(0, 10)]
87089
[(0, 10)]
87090
[(0, 8), (2205, 1), (2246, 1), (2491, 1)]
87114
[(0, 9), (4224, 1)]
87115
[(0, 9), (2205, 1), (2212, 1)]
87116
[(0, 7), (2217, 2), (2439, 1), (2432, 1)]
87124
[(2267, 6), (2091, 5), (2431, 5), (2452, 5), (2454, 5)]
87136
[(0, 4), (2246, 3), (4224, 2), (2439, 2), (2489, 2)]
87137
[(2246, 7), (0, 2), (2495, 2), (4224, 2), (2463, 2)]
87138
[(2246, 7), (0, 2), (2495, 2), (4224, 2), (2463, 2)]
87139
[(2246, 4), (2455, 3), (2458, 3), (0, 3), (2415, 2)]
87141
[(0, 7), (7517, 1), (2438, 1), (2478, 1), (2435, 1)]
87142
[(2415, 3), (2439, 3), (0, 3), (2456, 2), (2495, 2)]
87143
[(0, 3), (2439, 2), (2237, 2), (2246, 2), (2455, 1)]
87144
[(2246, 5), (0, 3), (2495, 2), (1618, 1), (2091, 1)]
87145
[(0, 5), (101046, 2), (2237, 2), (2435, 1), (1618, 1)]
87146
[(0, 8), (2079, 1), (2435, 1), (2456, 1), (2210, 1)]
87147
[(4224, 4), (2438, 4), (2246, 4), (2091, 1), (2431, 1)]
87150
[(0, 4), (2415, 2), (2439, 2), (2188, 1), (2237, 1)]
87153
[(

[(0, 8), (2237, 2), (2246, 2)]
88552
[(0, 8), (2237, 2), (2246, 2)]
88555
[(0, 10)]
88560
[(0, 9), (2438, 1), (1944, 1)]
88561
[(0, 10)]
88562
[(0, 8), (2237, 2), (2246, 2)]
88563
[(0, 10)]
88564
[(0, 10)]
88565
[(0, 10)]
88566
[(0, 10)]
88567
[(2091, 4), (2431, 4), (2452, 3), (2454, 3), (2217, 3)]
88569
[(0, 9), (2237, 1), (2246, 1)]
88570
[(0, 10)]
88572
[(0, 10)]
88573
[(0, 9), (2415, 1), (4224, 1), (2438, 1), (2439, 1)]
88574
[(0, 10)]
88575
[(0, 10)]
88576
[(0, 9), (2415, 1), (4224, 1), (2438, 1), (2439, 1)]
88580
[(0, 4), (2439, 3), (2415, 2), (2217, 2), (2076, 1)]
88583
[(0, 9), (4866, 1), (2246, 1)]
88584
[(0, 9), (2246, 1)]
88585
[(0, 10)]
88586
[(0, 9), (2237, 1)]
88587
[(0, 9), (2471, 1)]
88588
[(0, 10)]
88589
[(0, 10)]
88590
[(0, 7), (2471, 3), (2246, 1)]
88591
[(0, 10)]
88592
[(0, 8), (100529, 1), (2471, 1), (1964, 1)]
88593
[(0, 9), (2458, 1), (2246, 1)]
88594
[(0, 9), (1935, 1)]
88595
[(0, 8), (2432, 1), (7295, 1)]
88596
[(0, 7), (2205, 3), (2212, 3), (2091, 1), (2431, 1

In [187]:
file_category_results = open("ExtraProducts__category_TfIDFword2vec.txt","w")
count=0

extra_levels_products=[(extra_level1_products,products_level1),(extra_level2_products,products_level2),(extra_level3_products,products_level3),\
                       (extra_level4_products,products_level4),(extra_level5_products,products_level5)]
for level in extra_levels_products:
    for product in level[0]:
        nested_categories= [find_category(x,level[1]) for x in product_similar_dict[product]]
        flat_categories = [item for sublist in nested_categories for item in sublist]
       # print(flat_categories)
        most_commons= Counter(flat_categories).most_common(5)
        for m in most_commons:
            if m[1] > 5 and m[0]!=0:
                file_category_results.write("ProductID:{0}    CategoryID:{1}   Name:{2}              {3}\n".format(product,m[0],\
                category_data.loc[m[0]]['Name'],category_data.loc[m[0]]['d']))
                count+=1
file_category_results.close()

In [188]:
print(count)

2908


In [246]:
file_category_results = open("Products__category_word2vec.txt","w")
#file_category_matchOld= open("Products__category_wrod2vec_matchOld.txt","w")
count_all=0
count_matchOld=0
count_0=0

for product in product_attr.index.values.tolist():
    nested_categories= [find_category(x,products_OldCategories) for x in product_similar_dict[product]]
    flat_categories = [item for sublist in nested_categories for item in sublist]
   # print(flat_categories)
    most_commons= Counter(flat_categories).most_common(10)

    for m in most_commons:
        if (m[1] > 6 and m[0]!=0 and ~(((products_OldCategories['ProductID']==product) & (products_OldCategories['CategoryID']==m[0])).any())):
            file_category_results.write("ProductID:{0}    CategoryID:{1}   Name:{2}              {3}\n".format(product,m[0],\
            category_data.loc[m[0]]['Name'],category_data.loc[m[0]]['d']))
            count_all+=1
        if (m[1] > 4 and m[0]!=0 and (((products_OldCategories['ProductID']==product) & (products_OldCategories['CategoryID']==m[0])).any())):
            #file_category_matchOld.write("ProductID:{0}    CategoryID:{1}   Name:{2}              {3}\n".format(product,m[0],\
            #category_data.loc[m[0]]['Name'],category_data.loc[m[0]]['d']))
            count_matchOld+=1
file_category_results.close()
#file_category_matchOld.close()

In [240]:
print(count_all)
print(count_matchOld)
print(count_0)

4766
59917
356


In [241]:
count_matchOld

59917

In [242]:
print(products_OldCategories.nunique())

ProductID     29372
CategoryID     1533
level             5
d              1533
Name           1531
dtype: int64


In [243]:
 products_OldCategories.drop_duplicates().shape[0]

101616

In [104]:
query3='''
SELECT A.ProductID as ProductID, A.ParentCategoryID as CategoryID, B.level as level, B.d as d, B.Name as Name 
FROM `inventoryvelocity.input.product_parent_categories_relation` AS A
Inner join `rec-serv.AutoSuggestion.category_data` AS B on A.ParentCategoryID=B.CategoryID
WHERE B.CategoryID not in (1791,1738,1775,1739)
and B.ParentCategoryID in (101072)
'''
products_vitamins=pd.read_gbq(query3,p,dialect='standard')

products_vitamins=products_vitamins[products_vitamins['ProductID'].isin(product_attr.index.values)]
list_vitamin_products=products_vitamins['ProductID'].unique().tolist()

In [217]:

list_level2_products=products_level2['ProductID'].unique().tolist()
full_text_tokenized=[]
for index in list_level2_products:
#for index in list_vitamin_products:
    s=product_attr.loc[index]['DisplayName']
    s = re.sub(r'[^\w\s]',' ',s)
    #s = s.translate(None, string.punctuation)
    words = nltk.word_tokenize(s)
   # words = nltk.word_tokenize(product_attr.loc[index]['DisplayName'])
   # words=product_attr.loc[index]['DisplayName'].split()
    words = normalize(words)
   # words = [x for x in words if x != ' ']
    full_text_tokenized.append(words)
   # words1 = nltk.word_tokenize(product_attr.loc[index]['Description'])
   # words1 = normalize(words)
   # words = words + words1
   # full_text_tokenized.append(words)
    
    #full_text_tokenized.append (gensim.utils.simple_preprocess (product_attr.loc[index]['DisplayName']))
    #full_text_tokenized.append (gensim.utils.simple_preprocess (product_attr.loc[index]['Description']))
    
    #full_text_tokenized.append(product_attr.loc[index]['DisplayName'].split(" "))
model = gensim.models.Word2Vec(full_text_tokenized, size=50,sg=1,window=2,min_count=1)
model.train(full_text_tokenized, total_examples=len(full_text_tokenized), epochs=10)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))


model_TfidfEmbeddingVectorizer=TfidfEmbeddingVectorizer(w2v)
#model_TfidfEmbeddingVectorizer.fit(Full_text_list)
model_TfidfEmbeddingVectorizer.fit(product_attr['DisplayName'])
#model_TfidfEmbeddingVectorizer.fit(sentences)
tfidfEmbedding_matrix_allProducts=model_TfidfEmbeddingVectorizer.transform(full_text_tokenized)
print(tfidfEmbedding_matrix_allProducts.shape)
#cosine_similarities=np.zeros((tfidfEmbedding_matrix_allProducts.shape[0],tfidfEmbedding_matrix_allProducts.shape[0]))
cosine_similarities=cosine_similarity(tfidfEmbedding_matrix_allProducts,tfidfEmbedding_matrix_allProducts)

product_similarities=pd.DataFrame(cosine_similarities, index=list_level2_products,columns=list_level2_products)
#product_similarities=pd.DataFrame(cosine_similarities, index=list_vitamin_products,columns=list_vitamin_products)
#print(product_similarities)
product_similar_dict = dict()
for product in list_level2_products:
#for product in list_vitamin_products:
    product_similar_dict[product] = product_similarities.loc[product].sort_values(ascending=False).index.tolist()[1:11]

C:\Users\fjavadi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


(29573, 50)


In [214]:
print(len(products_vitamins))
print(products_vitamins.nunique())
products_vitamins['Name'].unique()
products_level2.nunique()
print(len(products_level2))

1302
ProductID     1291
CategoryID       5
level            1
d                5
Name             5
dtype: int64
50381


In [223]:
products_OldCategories=products_OldCategories [products_OldCategories['level']==2]
test_category_results = open("test__category_word2vec.txt","w")
count_vitamin=0
count_extra=0
count_0=0
product_error=0
total_products=0
for product in products_vitamins['ProductID'].values.tolist():
#for product in list_level2_products:
    nested_categories= [find_category(x,products_level2) for x in product_similar_dict[product]]
    flat_categories = [item for sublist in nested_categories for item in sublist]
   # print(flat_categories)
    most_commons= Counter(flat_categories).most_common(2)
    if(most_commons[0][0]==0):
        count_0+=1
    p=False
    for m in most_commons:
               
        if (m[1] > 5 and m[0]!=0 and ~(((products_OldCategories['ProductID']==product) & (products_OldCategories['CategoryID']==m[0])).any())):
            #file_category_results.write("ProductID:{0}    CategoryID:{1}   Name:{2}              {3}\n".format(product,m[0],\
            #category_data.loc[m[0]]['Name'],category_data.loc[m[0]]['d']))
            count_extra+=1
        if (m[1] > 4 and m[0]!=0 and (((products_OldCategories['ProductID']==product) & (products_OldCategories['CategoryID']==m[0])).any())):
            #file_category_matchOld.write("ProductID:{0}    CategoryID:{1}   Name:{2}              {3}\n".format(product,m[0],\
            #category_data.loc[m[0]]['Name'],category_data.loc[m[0]]['d']))
            count_vitamin+=1
            p=True
    if(p==True):
        total_products+=1
    if(p==False):
        test_category_results.write("ProductID:{0}\n".format(product))
        product_error+=1
        print("ProductID:{0}      {1}".format(product,product_attr.loc[product]['DisplayName']))
            
#file_category_results.close()


ProductID:47350      Lung, Bronchial & Sinus Health, 45 Tablets
ProductID:81132      Teen Focus, Advanced Formula for Improved Focus, 90 Capsules
ProductID:81136      Immunessence, Premium Immune System Support, 120 Vegetarian Capsules
ProductID:82097      DHEA Plus 15, Natural DHEA Cream, Unscented, 2 oz (57 g)
ProductID:82398      LiverCare Powder, 30 Packets, 0.14 oz (4 g) Each
ProductID:86015      Anti-Alcohol, HepatoProtection Complex, 60 Vegetarian Capsules
ProductID:1465      Pregnenolone, 50 mg, 120 Tablets
ProductID:81096      Sambu Guard, 5.9 fl oz (175 ml)
ProductID:81097      Sambu Guard for Kids, 5.9 fl oz (175 ml)
ProductID:81098      Nasturtium Capsules, With Rosehip, Elderflower and Vitamin C, 60 Vegan Capsules
ProductID:81099      Linden Blossom, 5.9 fl oz (175 ml)
ProductID:82191      Coconut Cooking Oil, Unflavored, 16 fl oz (473 ml)
ProductID:82317      Thyro Support, 90 Vegan Capsules
ProductID:82409      Natural Activated Charcoal, 260 mg, 20 Packets, 2 Capsules E

ProductID:88817      Prash, Tonic Ambrosia, 5 oz (144 g)
ProductID:73759      GABA, Cherry, 25 mg , 120 Micro Tablets
ProductID:73775      GABA L-Theanine Stress B Lozenge, Natural Mango Tangerine Flavor, 100 Lozenges
ProductID:4048      DHEA, 25 mg, 100 Dissolve in Mouth Tablets
ProductID:4104      Herbal Expec, Natural Cherry Flavor, 8.8 fl oz (260 ml)
ProductID:4220      Echinacea Zinc, Herbalozenge, Very Cherry, 15 Lozenges
ProductID:4226      Orange C, Herbalozenge, Zesty Orange, 15 Lozenges
ProductID:4227      Lemon Zinc, Herbalozenge, Natural Lemon Flavor, 15 Lozenges
ProductID:4260      Elderberry Zinc, Herbalozenge, Sweet Elderberry, 15 Lozenges
ProductID:4273      Colon Clenz, 120 Vegetarian Capsules
ProductID:4499      Lung-Saver, 60 Capsules
ProductID:4707      DefensePlus, Maximum Strength, 250 mg, 45 Vegan Tablets
ProductID:4789      Great Plains, Bentonite, Detox, 16 fl oz (473 ml)
ProductID:4835      Liver X with BioSorb, 60 Vegetarian Capsules
ProductID:4845      Ultra

ProductID:78031      Pure Synergy, Super Pure Milk Thistle Organic Extract, 60 Organic Vegetarian Caps
ProductID:78122      Immune Support & Vitamin C Drink Mix, Natural Orange Flavor, 10 Packets, 3.5 oz (99 g)
ProductID:78127      Cold Crush for Children, Age 3+, Organic Cold & Cough, Organic Acai Berry Flavor, 60 Chewable Tablets
ProductID:78193      Phosphatidyl Serine, Soy-Free, 120 mg , 60 Tablets
ProductID:2169      DMG-B15-Plus, Energy, 60 Veggie Caps
ProductID:2176      Fatigued to Fantastic!, Energy Revitalization System, Berry Splash Flavor, 1.3 lbs (609 g)
ProductID:18473      Iodine & Tyrosine, 60 Capsules
ProductID:18646      Organic Liver and Gallbladder Support, Whole Food POWder, 3.2 oz (90 g)
ProductID:18696      Moducare, 90 Capsules
ProductID:18708      Diabenil, 90 Vegetarian Capsules
ProductID:18735      Cortrex, 60 Capsules
ProductID:18737      Arabinex, Arabinogalactans, 3.5 oz (100 g)
ProductID:18741      PharmaGABA-100, 60 Capsules
ProductID:18960      D-Mannos

ProductID:54509      RememBrain, Cognitive Health, 60 Veg Capsules
ProductID:54808      Saffron Extract, 50 Veggie Caps
ProductID:54822      Total Colon Wellness, 60 Tablets
ProductID:54873      Coldcalm, Children's Cold Relief, 2 Tubes, Approx 80 Pellets Per Tube
ProductID:55233      PectaSol-C, Modified Citrus Pectin, 90 Vegetable Capsules
ProductID:55600      Mannose Cranberry, 90 Veg Capsules
ProductID:55652      Tri Sugar Shield, 60 Vegetarian Capsules
ProductID:55653      Cytokine Suppress with EGCG, 30 Vegetarian Capsules
ProductID:55814      Kidz, Fruit Punch, 30 Packets, 9.7 oz (276 g)
ProductID:56504      Argentyn 23, 16 fl oz (473 ml)
ProductID:5020      GABA, 750 mg, 100 Veg Capsules
ProductID:5022      Super Cortisol Support, 90 Veg Capsules
ProductID:5049      U.T. Vibrance, D-Mannose 5,000 mg, Version 1.1, 2.28 oz (64.55 g)
ProductID:5151      Neuro-Max II, 60 Vegan Capsules
ProductID:56886      Sunflower Liquid Lecithin, 16 fl oz (473 ml)
ProductID:82934      DHEA Micro

ProductID:21505      Source of Life, Immune Booster, 90 Tablets
ProductID:21548      Tao in a Bottle, 450 mg, 60 Capsules
ProductID:21602      Diamond Mind, 500 mg Each, 100 Veggie Caps
ProductID:21612      Diamond Mind, Super Potency Extract, 2 fl oz (60 ml)
ProductID:22232      NADH, 10 mg, 60 Veg Capsules
ProductID:22307      Olbas Lozenges, Sugar Free, Black Currant Flavor, 24 Lozenges
ProductID:22413      Phosphatidylserine, 200 mg, 60 Softgels
ProductID:22662      Super Kids Throat Spray, Peppermint, 1 fl oz (30 ml)
ProductID:23135      AnxioCalm, 45 Tablets
ProductID:23321      NattoZyme, 100 mg, 180 Softgels
ProductID:23429      Solar, Energy Gum, Big Bang Bubble, 50 Pieces
ProductID:23469      Echinacea/Eyebright, 2 fl oz (59 ml)
ProductID:23707      Prevagen, Regular Strength, 30 Capsules
ProductID:23744      Phosphatidylserine with SerinAid, 100 mg, 120 Veggie Caps
ProductID:24580      Ox Bile, 125 mg, 180 Vegicaps
ProductID:24736      DHEA 100, 100 mg, 60 Veggie Caps
Produc

...

ProductID:86009      Organic Greens, Alkalize & Detox, 7.70 oz (220.35 g)
ProductID:82242      Fermented Broccoli Sprouts, 30 Capsules
ProductID:1074      Chlorella, 500 mg, 200 Tablets
ProductID:497      Chlorella, 400 mg, 100 Veg Capsules
ProductID:498      Chlorophyll, 100 mg, 90 Veggie Caps
ProductID:685      Kelp, 150 mcg, 200 Tablets
ProductID:800      Natural Spirulina, 500 mg, 120 Veg Capsules
ProductID:22429      Super Acai, Brazilian Berry, 50 Softgels
ProductID:82546      Lucuma Powder, The Gold Of The Incas, 16 oz (454 g)
ProductID:83697      Field Fresh, Organic Grasses & Ancient Grains Master Blend, Natural Flavor, 6.35 oz (180 g)
ProductID:83983      Clean Green Protein with Lentein, Vanilla Chai, 1.57 lbs (710 g)
ProductID:84827      Organic, Coco Gogo Smoothie Mix, Chocolate Vitality Boost, 6 oz (170 g)
ProductID:2456      Broccoli Sprouts Extract, 60 Tablets
ProductID:84598      Kale Powder, 8 oz (226.8 g)
ProductID:1835      Chlorella, Micro-Algae, 410 mg, 100 Vegeta

ProductID:50425      Broken Cell Wall Chlorella Tablets, 250 mg, 228 Tablets, 2 oz (57 g)
ProductID:50427      Himalayan Shilajit Powder, 3.5 oz (100 g)
ProductID:51748      Sun Is Shining Supergreens, 8 oz (227 g)
ProductID:52312      Broccoli Sprouts, 250 mg, 100 Veggie Caps
ProductID:52810      Chlorofresh, Chlorophyll Drops, Mint Flavored, 2 fl oz (59 ml)
ProductID:53525      Arriba Nacional Cacao Paste, 8 oz (226.8 g)
ProductID:59381      Organic Cacao Powder, 1 lb (454 g)
ProductID:59390      Organic Cacao Paste, 1 lb (454 g)
ProductID:59392      Amazon Acai Powder, 4 oz (113 g)
ProductID:59947      Organic Raw Whole Cacao Beans, 16 oz (453 g)
ProductID:62187      Organic Blue-Green Algae, 90 Veggie Caps
ProductID:64432      Organic Cacao Nibs Heirloom, 8 oz (227 g)
ProductID:69778      Camu Camu, 60 Vegetable Caps
ProductID:70169      Liquid Chlorophyll, Unflavored, 100 mg, 16.2 fl oz (480 ml)
ProductID:72126      Organic Ultra Juice Green Powder, 0.66 lb (300 g)
ProductID:72127

ProductID:45934      Nordic Omega-3 Fishies, Yummy Tutti Frutti Taste, 300 mg, 36 Fishies
ProductID:47794      Baby, Liquid Vitamin D3, 400 IU, 0.08 fl oz (2.5 ml), 90 Drops
ProductID:49249      Complete Probiotics Powder Packets for Kids, Natural Raspberry Flavor, 30 Packets, 0.12 oz (3.5 g) Each
ProductID:50054      Chewable Cod Liver Oil, with Vitamins A, C, & D, Orange Flavor, 100 Tablets
ProductID:51848      Organic Vitamin D3 Drops, Natural Berry Flavor, 400 IU, 0.338 fl oz (10 ml)
ProductID:55858      Kid's Chewable Calcium, Natural Vanilla Flavor, 250 mg, 120 Tablets
ProductID:59369      Probiotic + Prebiotic Fiber, Sugar-Free Milk Chocolate, 40 Bears
ProductID:63260      Just 4 Kids! Potent Probiotics with Organic Prebiotics Powder, Banana, 2 oz (57 g)
ProductID:63262      Just for Kids! Probiotic Chocolate Hearts, 30 Hearts, 2 oz (56 g)
ProductID:66774      Kids, Propolis Astragalus, Throat Spray, 1 fl oz (30 ml)
ProductID:67331      Infants & Kids, Liquid Vitamin D3, 400 IU,

ProductID:56864      Healthy Hair with Biotin 10,000 Plus, 60 Veggie Caps
ProductID:58892      Biotin Bursts, Brazilian Acai Berry, 30 Soft Chews
ProductID:65006      Acne Essentials, 60 Capsules
ProductID:71365      Beef Liver, 750 mg, 120 Capsules
ProductID:72129      Biotin, Sustained Release, 90 Tablets
ProductID:72340      Certified Organic Biotin, 5000 mcg, 90 Organic Vegetarian Tablets
ProductID:72977      Longer Stronger Hair & Nails, 60 Liquid Soft-Gels
ProductID:75650      Biotin Drops, For Healthy Hair & Nails, Natural Vanilla Flavor, 10,000 mcg, 2 fl oz (60 ml)
ProductID:79101      Biotin Gummies, Strawberry Flavor, 10,000 mcg, 60 Gummies
ProductID:81003      Iron Beard, 60 Veggie Capsules
ProductID:83419      Extra Strength Hair Formula, 60 Tablets
ProductID:83705      Beauty Booster, Natural Berry Flavor, 2.96 oz (84 g)
ProductID:85299      Regenotin, Advanced Collagen Generator, 60 Vegetarian Capsules
ProductID:86394      Ultra Biotin, Natural Mixed Berry Flavor, 10,000 

ProductID:88695      Organic Deodorant, Sweet Orange, 2.5 oz (70.8 g)


In [222]:
print(count_vitamin)
print(count_extra)
print(count_0)
print(product_error)
print(total_products)

25166
3531
0
9797
19776


In [220]:


nested_categories= [find_category(x,products_vitamins) for x in product_similar_dict[24625]]
print(product_similar_dict[82191])

[75439, 79966, 80607, 84071, 71107, 12929, 84550, 76795, 6611, 10302]


In [125]:
s=product_attr.loc[23181]['DisplayName']
s = re.sub(r'[^\w\s]',' ',s)
words = nltk.word_tokenize(s)
words = normalize(words)

words = [x for x in words if x != ' ']
words

['vitamin', 'e', 'complex', 'licaps']

In [332]:
words = nltk.word_tokenize(product_attr.loc[17519]['DisplayName'])
words = normalize(words)
words

['vitamin', 'k', ' ', ' ', 'mcg', ' ', ' ', 'tablet']